**Check for Java**

In [2]:
!java --version

java 16.0.1 2021-04-20
Java(TM) SE Runtime Environment (build 16.0.1+9-24)
Java HotSpot(TM) 64-Bit Server VM (build 16.0.1+9-24, mixed mode, sharing)


**Import and Check env for Tabula**

In [1]:
import tabula

In [2]:
tabula.environment_info()

Python version:
    3.8.5 (default, Jul 21 2020, 10:48:26) 
[Clang 11.0.3 (clang-1103.0.32.62)]
Java version:
    java version "16.0.1" 2021-04-20
Java(TM) SE Runtime Environment (build 16.0.1+9-24)
Java HotSpot(TM) 64-Bit Server VM (build 16.0.1+9-24, mixed mode, sharing)
tabula-py version: 2.2.0
platform: macOS-10.15.7-x86_64-i386-64bit
uname:
    uname_result(system='Darwin', node='Claytons-MBP.home', release='19.6.0', version='Darwin Kernel Version 19.6.0: Mon Apr 12 20:57:45 PDT 2021; root:xnu-6153.141.28.1~1/RELEASE_X86_64', machine='x86_64', processor='i386')
linux_distribution: ('Darwin', '19.6.0', '')
mac_ver: ('10.15.7', ('', '', ''), 'x86_64')
    


---------------

# What is Tabula?

"tabula-py is just a wrapper of tabula-java , make sure you’ve installed Java and you can use java command on your terminal. Many issue reporters forget to set PATH for java command.

You can check whether tabula-py can call java from Python process with tabula.environment_info() function." [FAQ](https://tabula-py.readthedocs.io/en/latest/faq.html)

In [603]:
# I am only taking the 3rd page because it is where the tables start
# Toggling the lattice parameter produces different outputs in the organization of the df
# I don't see any difference when toggling the stream parameter
# We are leaving stream = False and putting lattice = True
# It is default I think but multple_tables is set to True 
pdf_path = '../data/Medical Surgical.pdf'
dfs = tabula.read_pdf(pdf_path, pages='3',lattice=True, stream=False, multiple_tables=True)

In [604]:
type(dfs)

list

In [605]:
len(dfs)

6

In [606]:
dfs

[Empty DataFrame
 Columns: [Traditional Wound Care]
 Index: [],
 Empty DataFrame
 Columns: [Traditional Wound Care]
 Index: [],
         Sterile       Unnamed: 0                Unnamed: 1
 0         C5119   2” x 2”, 8-ply  2/pk, 50 pk/bx, 30 bx/cs
 1         C5129  3” x 3”, 12-ply  2/pk, 50 pk/bx, 24 bx/cs
 2         C5189   4” x 4”, 8-ply  2/pk, 50 pk/bx, 12 bx/cs
 3         C5139  4” x 4”, 12-ply  2/pk, 25 pk/bx, 24 bx/cs
 4          C517  8” x 4”, 12-ply  2/pk, 25 pk/bx, 20 bx/cs
 5   Non-Sterile              NaN                       NaN
 6        C82082   2” x 2”, 8-ply          200/bg, 25 bg/cs
 7        C82122  2” x 2”, 12-ply          200/bg, 40 bg/cs
 8        C83083   3” x 3”, 8-ply          200/bg, 20 bg/cs
 9        C83082  3” x 3”, 12-ply          200/bg, 20 bg/cs
 10       C84088   4” x 4”, 8-ply          200/bg, 20 bg/cs
 11       C84124  4” x 4”, 12-ply          200/bg, 10 bg/cs
 12       C84166  4” x 4”, 16-ply          200/bg, 10 bg/cs,
         Sterile        Unnamed

In [607]:
basicCareGauzeSponges = dfs[5]

In [608]:
basicCareGauzeSponges

,Sterile,Unnamed: 0,Unnamed: 1
0,8500,"2” x 2”, 8-ply","2/pk, 50 pk/bx, 30 bx/cs"
1,8501,"3” x 3”, 12-ply","2/pk, 50 pk/bx, 24 bx/cs"
2,8502,"4” x 4”, 8-ply","2/pk, 50 pk/bx, 12 bx/cs"
3,8503,"4” x 4”, 12-ply","2/pk, 25 pk/bx, 24 bx/cs"
4,Non-Sterile,NaN,NaN
5,8505,"2” x 2”, 8-ply","200/bg, 25 bg/cs"
6,8506,"2” x 2”, 12-ply","200/bg, 40 bg/cs"
7,8507,"3” x 3”, 8-ply","200/bg, 20 bg/cs"
8,8508,"3” x 3”, 12-ply","200/bg, 20 bg/cs"
9,8509,"4” x 4”, 8-ply","200/bg, 20 bg/cs"


In [609]:
# List of desired column names created outside of the function....
# Should have the same number of names as number of columns to be replaced
new_column_names = ['Item_#', 'Description', 'Packaging_Qty']

# Make this the function `rename_columns`
for count, col in enumerate(basicCareGauzeSponges.columns):
    basicCareGauzeSponges.rename(columns={col:new_column_names[count]}, inplace=True)

# Make this the function of counting Sterile items
for counter, row in enumerate(basicCareGauzeSponges.values):
    if row[0] == 'Non-Sterile':
        break

# Here, drops the non-sterile row and resets index....
# before counting non-sterile items
# May be irregularly used
basicCareGauzeSponges.dropna(inplace=True)
basicCareGauzeSponges.reset_index(drop=True, inplace=True)

# The counter is a variable from before that should show the number of Sterile Items
# Number of non-sterile items based on remaining number of rows in table
yes = counter
no = len(basicCareGauzeSponges) - counter 

# Create the list for the column 'Sterile'
sterile = ['Yes'] * yes + ['No'] * no

# Insert the Sterile list
basicCareGauzeSponges.insert(2, "Sterile", sterile)

basicCareGauzeSponges

# Need to cycle through the dataframe to extract each table and treat the same way

,Item_#,Description,Sterile,Packaging_Qty
0,8500,"2” x 2”, 8-ply",Yes,"2/pk, 50 pk/bx, 30 bx/cs"
1,8501,"3” x 3”, 12-ply",Yes,"2/pk, 50 pk/bx, 24 bx/cs"
2,8502,"4” x 4”, 8-ply",Yes,"2/pk, 50 pk/bx, 12 bx/cs"
3,8503,"4” x 4”, 12-ply",Yes,"2/pk, 25 pk/bx, 24 bx/cs"
4,8505,"2” x 2”, 8-ply",No,"200/bg, 25 bg/cs"
5,8506,"2” x 2”, 12-ply",No,"200/bg, 40 bg/cs"
6,8507,"3” x 3”, 8-ply",No,"200/bg, 20 bg/cs"
7,8508,"3” x 3”, 12-ply",No,"200/bg, 20 bg/cs"
8,8509,"4” x 4”, 8-ply",No,"200/bg, 20 bg/cs"
9,8510,"4” x 4”, 12-ply",No,"200/bg, 10 bg/cs"


### This is a proof of concept that the steps work: The MVP

-----
# Now lets work to make this flexible by encapsulating it into functions

In [763]:
import re

In [800]:
# This is a RESET button 
pdf_path = '../data/Medical Surgical.pdf'
dfs = tabula.read_pdf(pdf_path, pages='5',lattice=True, stream=False, multiple_tables=True)

In [801]:
dfs

[Empty DataFrame
 Columns: [Traditional Wound Care]
 Index: [],
 Empty DataFrame
 Columns: [Traditional Wound Care]
 Index: [],
    Sterile                                       Unnamed: 0  \
 0     7265                     I.V. Sponge,\r2” x 2”, 6-ply   
 1     7470                 T-Drain\rSponge,\r4” x 4”, 6-ply   
 2     7468  T-Drain\rSponge, 100%\rCotton,\r4” x 4”, 16-ply   
 
                  Unnamed: 1  
 0  2/pk, 35 pk/bx, 20 bx/cs  
 1  2/pk, 25 pk/bx, 12 bx/cs  
 2  1/pk, 25 pk/bx, 12 bx/cs  ,
    Sterile      Unnamed: 0                Unnamed: 1
 0     8746  4” x 4”, 6-ply  2/pk, 25 pk/bx, 12 bx/cs,
    Sterile Unnamed: 0                Unnamed: 1
 0     9550  1” Sphere  1/pouch, 50/bx, 10 bx/cs
 1     9552  1” Sphere  2/pouch, 50/bx, 10 bx/cs,
   Sterile       Unnamed: 0                Unnamed: 1
 0     841  1-5/8” x 2-5/8”  1/pk, 50 pk/bx, 12 bx/cs
 1    841B  1-5/8” x 2-5/8”          1/pk, 1000 pk/cs,
    Sterile Unnamed: 0                Unnamed: 1
 0     5434    4” x 

In [796]:
# This takes a list of df tables
def drop_empty_tables(df_list):
    new_df = []
    for table in df_list:
        if table.empty == True:  # for some reason dfs.remove(table) only removed first instance
            continue
            #dfs.remove(table)
        else:  # We will create a new list instead with the filled tables
            new_df.append(table)
    return new_df


#new_df

In [784]:
def rename_columns(new_df):
    for table in new_df:    
    # now this is going to rename all the columns the select names
        for count, col in enumerate(table.columns):
            table.rename(columns={col:new_column_names[count]}, inplace=True)
#new_df
            

In [767]:
for table in new_df:    
    # now this is going to rename all the columns the select names
    for count, col in enumerate(table.columns):
        table.rename(columns={col:new_column_names[count]}, inplace=True)
#new_df

In [785]:
def create_sterile_column(new_df):
    for table in new_df:    
        for counter, row in enumerate(table.values):
            if row[0] == 'Non-Sterile':
                break

        table.dropna(inplace=True)
        table.reset_index(drop=True, inplace=True)

        yes = counter
        no = len(table) - counter 

        sterile = ['Yes'] * yes + ['No'] * no

        table.insert(2, "Sterile", sterile)

In [802]:
new_df = drop_empty_tables(dfs)

In [803]:
rename_columns(new_df)

In [804]:
create_sterile_column(new_df)

In [807]:
new_df[0]['Description']

0                       I.V. Sponge,\r2” x 2”, 6-ply
1                   T-Drain\rSponge,\r4” x 4”, 6-ply
2    T-Drain\rSponge, 100%\rCotton,\r4” x 4”, 16-ply
Name: Description, dtype: object

In [768]:
# This needs to be modified because it will not catch tables that don't start with sterile items        
for table in new_df:    
    for counter, row in enumerate(table.values):
        if row[0] == 'Non-Sterile':
            break
    
    table.dropna(inplace=True)
    table.reset_index(drop=True, inplace=True)
    
    yes = counter
    no = len(table) - counter 

    sterile = ['Yes'] * yes + ['No'] * no
    
    table.insert(2, "Sterile", sterile)


In [769]:
new_df

[    Item_#      Description Sterile             Packaging_Qty
 0    C5119   2” x 2”, 8-ply     Yes  2/pk, 50 pk/bx, 30 bx/cs
 1    C5129  3” x 3”, 12-ply     Yes  2/pk, 50 pk/bx, 24 bx/cs
 2    C5189   4” x 4”, 8-ply     Yes  2/pk, 50 pk/bx, 12 bx/cs
 3    C5139  4” x 4”, 12-ply     Yes  2/pk, 25 pk/bx, 24 bx/cs
 4     C517  8” x 4”, 12-ply     Yes  2/pk, 25 pk/bx, 20 bx/cs
 5   C82082   2” x 2”, 8-ply      No          200/bg, 25 bg/cs
 6   C82122  2” x 2”, 12-ply      No          200/bg, 40 bg/cs
 7   C83083   3” x 3”, 8-ply      No          200/bg, 20 bg/cs
 8   C83082  3” x 3”, 12-ply      No          200/bg, 20 bg/cs
 9   C84088   4” x 4”, 8-ply      No          200/bg, 20 bg/cs
 10  C84124  4” x 4”, 12-ply      No          200/bg, 10 bg/cs
 11  C84166  4” x 4”, 16-ply      No          200/bg, 10 bg/cs,
       Item_#       Description Sterile             Packaging_Qty
 0       6208   2” x 2”,\r8-ply     Yes  2/pk, 50 pk/bx, 30 bx/cs
 1       6312   3” x 3”,\r2-ply     Yes  2/pk, 5

In [808]:
prob_df = new_df[0]

In [809]:
clean_description = []
for val in prob_df['Description']:
    clean_one = re.sub(r'\s', ' ',val)
    clean_description.append(clean_one)

In [810]:
new_df[0]['Description']=clean_description

In [811]:
new_df

[   Item_#                                   Description Sterile  \
 0    7265                   I.V. Sponge, 2” x 2”, 6-ply     Yes   
 1    7470                T-Drain Sponge, 4” x 4”, 6-ply     Yes   
 2    7468  T-Drain Sponge, 100% Cotton, 4” x 4”, 16-ply      No   
 
               Packaging_Qty  
 0  2/pk, 35 pk/bx, 20 bx/cs  
 1  2/pk, 25 pk/bx, 12 bx/cs  
 2  1/pk, 25 pk/bx, 12 bx/cs  ,
    Item_#     Description Sterile             Packaging_Qty
 0    8746  4” x 4”, 6-ply      No  2/pk, 25 pk/bx, 12 bx/cs,
    Item_# Description Sterile             Packaging_Qty
 0    9550   1” Sphere     Yes  1/pouch, 50/bx, 10 bx/cs
 1    9552   1” Sphere      No  2/pouch, 50/bx, 10 bx/cs,
   Item_#      Description Sterile             Packaging_Qty
 0    841  1-5/8” x 2-5/8”     Yes  1/pk, 50 pk/bx, 12 bx/cs
 1   841B  1-5/8” x 2-5/8”      No          1/pk, 1000 pk/cs,
    Item_# Description Sterile             Packaging_Qty
 0    5434     4” x 3”     Yes  2/pk, 25 pk/bx, 24 bx/cs
 1    54

Because we are going to have to modify a number of factors each time we run this then we need to have the functions work on only one element and the write a for loop outside of that function to iterate through....

We are going to iterate through table of the df object

--------
### Focus on detecting the sterile/non-sterile tags

We have to look at the columns of each table because that is where the tag appears in the output.


In [759]:
# This is a RESET button 
pdf_path = '../data/Medical Surgical.pdf'
dfs = tabula.read_pdf(pdf_path, pages='3',lattice=True, stream=False, multiple_tables=True)

In [ ]:
new_df = []
for table in dfs:
    if table.empty == True:  # for some reason dfs.remove(table) only removed first instance
        continue
        #dfs.remove(table)
    else:  # We will create a new list instead with the filled tables
        new_df.append(table)

In [760]:
for table in dfs:    
    # now this is going to rename all the columns the select names
    for count, col in enumerate(table.columns):
        table.rename(columns={col:new_column_names[count]}, inplace=True)

In [761]:
for table in dfs:
    if table.columns[0] == 'Sterile':   
        for counter, row in enumerate(table.values):
            if row[0] == 'Non-Sterile':
                break

        table.dropna(inplace=True)
        table.reset_index(drop=True, inplace=True)

        yes = counter
        no = len(table) - counter 

        sterile = ['Yes'] * yes + ['No'] * no

        table.insert(2, "Sterile", sterile)

        
    

In [762]:
dfs

[Empty DataFrame
 Columns: [Item_#]
 Index: [],
 Empty DataFrame
 Columns: [Item_#]
 Index: [],
          Item_#      Description             Packaging_Qty
 0         C5119   2” x 2”, 8-ply  2/pk, 50 pk/bx, 30 bx/cs
 1         C5129  3” x 3”, 12-ply  2/pk, 50 pk/bx, 24 bx/cs
 2         C5189   4” x 4”, 8-ply  2/pk, 50 pk/bx, 12 bx/cs
 3         C5139  4” x 4”, 12-ply  2/pk, 25 pk/bx, 24 bx/cs
 4          C517  8” x 4”, 12-ply  2/pk, 25 pk/bx, 20 bx/cs
 5   Non-Sterile              NaN                       NaN
 6        C82082   2” x 2”, 8-ply          200/bg, 25 bg/cs
 7        C82122  2” x 2”, 12-ply          200/bg, 40 bg/cs
 8        C83083   3” x 3”, 8-ply          200/bg, 20 bg/cs
 9        C83082  3” x 3”, 12-ply          200/bg, 20 bg/cs
 10       C84088   4” x 4”, 8-ply          200/bg, 20 bg/cs
 11       C84124  4” x 4”, 12-ply          200/bg, 10 bg/cs
 12       C84166  4” x 4”, 16-ply          200/bg, 10 bg/cs,
          Item_#       Description             Packaging_Qty
 0

[Empty DataFrame
 Columns: [Traditional Wound Care]
 Index: [],
 Empty DataFrame
 Columns: [Traditional Wound Care]
 Index: [],
     Item_#      Description Sterile             Packaging_Qty
 0    C5119   2” x 2”, 8-ply     Yes  2/pk, 50 pk/bx, 30 bx/cs
 1    C5129  3” x 3”, 12-ply     Yes  2/pk, 50 pk/bx, 24 bx/cs
 2    C5189   4” x 4”, 8-ply     Yes  2/pk, 50 pk/bx, 12 bx/cs
 3    C5139  4” x 4”, 12-ply     Yes  2/pk, 25 pk/bx, 24 bx/cs
 4     C517  8” x 4”, 12-ply     Yes  2/pk, 25 pk/bx, 20 bx/cs
 5   C82082   2” x 2”, 8-ply      No          200/bg, 25 bg/cs
 6   C82122  2” x 2”, 12-ply      No          200/bg, 40 bg/cs
 7   C83083   3” x 3”, 8-ply      No          200/bg, 20 bg/cs
 8   C83082  3” x 3”, 12-ply      No          200/bg, 20 bg/cs
 9   C84088   4” x 4”, 8-ply      No          200/bg, 20 bg/cs
 10  C84124  4” x 4”, 12-ply      No          200/bg, 10 bg/cs
 11  C84166  4” x 4”, 16-ply      No          200/bg, 10 bg/cs,
       Item_#      Description Sterile             P

In [676]:
# This is a proceedural function with no output
# It takes a single table and makes changes inplace
def rename_columns(table):
    for count, col in enumerate(table.columns):
        table.rename(columns={col:new_column_names[count]}, inplace=True)
            
# This is a proceedural function with no output.
# It takes a single table and makes changes inplace
def dropna_and_reset_index(table):
    table.dropna(inplace=True)
    table.reset_index(drop=True, inplace=True)

# This needs to return a list 
# It takes a single table
def sterile_item_counter(table):
    for counter, row in enumerate(table.values):
        if row[0] == 'Non-Sterile':
            break
    # calls our defined function            
    table.dropna(inplace=True)
    table.reset_index(drop=True, inplace=True)
    
    yes = counter
    no = len(table) - counter 
    
    sterile = ['Yes'] * yes + ['No'] * no
    
    table.insert(2, "Sterile", sterile)
    

In [677]:
drop_empty_tables(dfs)

[         Item_#      Description             Packaging_Qty
 0         C5119   2” x 2”, 8-ply  2/pk, 50 pk/bx, 30 bx/cs
 1         C5129  3” x 3”, 12-ply  2/pk, 50 pk/bx, 24 bx/cs
 2         C5189   4” x 4”, 8-ply  2/pk, 50 pk/bx, 12 bx/cs
 3         C5139  4” x 4”, 12-ply  2/pk, 25 pk/bx, 24 bx/cs
 4          C517  8” x 4”, 12-ply  2/pk, 25 pk/bx, 20 bx/cs
 5   Non-Sterile              NaN                       NaN
 6        C82082   2” x 2”, 8-ply          200/bg, 25 bg/cs
 7        C82122  2” x 2”, 12-ply          200/bg, 40 bg/cs
 8        C83083   3” x 3”, 8-ply          200/bg, 20 bg/cs
 9        C83082  3” x 3”, 12-ply          200/bg, 20 bg/cs
 10       C84088   4” x 4”, 8-ply          200/bg, 20 bg/cs
 11       C84124  4” x 4”, 12-ply          200/bg, 10 bg/cs
 12       C84166  4” x 4”, 16-ply          200/bg, 10 bg/cs,
          Item_#       Description             Packaging_Qty
 0          6208   2” x 2”,\r8-ply  2/pk, 50 pk/bx, 30 bx/cs
 1          6312   3” x 3”,\r2-ply  2

In [678]:
for table in dfs:
    rename_columns(table)

In [679]:
for table in dfs:
    sterile_item_counter(table)

UnboundLocalError: local variable 'counter' referenced before assignment

In [637]:
drop_empty_tables(dfs)

for table in dfs:
    rename_columns(table)

for table in dfs:
    sterile_item_counter(table)

UnboundLocalError: local variable 'counter' referenced before assignment

----------
# Let's try to combine libraries: Tabula + Fitz

In [315]:
import fitz

In [316]:
pdf_path = '../data/Medical Surgical.pdf'
doc = fitz.open(pdf_path)